In [1]:
import os
import sys
from common import commons
home = commons.home
extra_storage = commons.extra_storage
from features_preprocess import BED_binning
#from features_preprocess import BED_Preprocess,CADD_Preprocess_bedtools as CADD_Preprocess,DANN_Preprocess_bedtools as DANN_Preprocess,Eigen_Preprocess_bedtools as Eigen_Preprocess,GenoCanyon_Preprocess,WGBS_preprocess,GWAVA_Preprocess
from features_preprocess import BED_Preprocess,CADD_Preprocess,DANN_Preprocess,Eigen_Preprocess,GenoCanyon_Preprocess,WGBS_preprocess,GWAVA_Preprocess
import subprocess
import pandas as pd
from features_preprocess import get_winid
import prediction_commons
import numpy as np
import re
from pyliftover import LiftOver
from features_preprocess import WGBS_preprocess
from common import DataScaler as ds
from common.commons import rename_features
import gc
from pybedtools import BedTool

In [2]:
def wgbs_sites_selection(tss,allsites):
    tss = tss.sort_values(['chr','coordinate'])
    allsites = all_sites.sort_values(['chr','coordinate'])
    i = 0
    selected_sites = []
    #selected_sites = pd.DataFrame(columns=['chr','coordinate','tss_coordinate'])
    tss['before'] = tss['coordinate']-100000
    tss['after'] = tss['coordinate']+100000
    for row in allsites.iterrows():
        if i >= len(tss):
            break
        chr = row[1]['chr']
        coordinate = row[1]['coordinate']
        winid = row[1]['winid']
        if chr==tss.ix[i,'chr'] and coordinate>=tss.ix[i,'before'] and coordinate<=tss.ix[i,'after']:
            selected_sites.extend([[winid,chr,coordinate,tss.ix[i,'chr'],tss.ix[i,'coordinate']]])
        else:
            while  i<len(tss) and (chr>tss.ix[i,'chr'] or (chr==tss.ix[i,'chr'] and coordinate>tss.ix[i,'after'])):
                i += 1
            if i<len(tss) and chr==tss.ix[i,'chr'] and coordinate>=tss.ix[i,'before'] and coordinate<=tss.ix[i,'after']:
                selected_sites.extend([[winid,chr,coordinate,tss.ix[i,'chr'],tss.ix[i,'coordinate']]])
    df = pd.DataFrame(selected_sites,columns=['winid','chr','coordinate','tss_chr','tss_coordinate'])
    df['chr'] = df['chr'].astype('i8')
    return df


def nearest_tss(tss,sites_df):
    merged = pd.merge(sites_df,tss,how='outer',on=['chr','coordinate'])
    merged.sort_values(['chr','coordinate'],inplace=True)
    merged.rename(columns={'strand':'before_tss'},inplace=True)
    merged.ix[merged['before_tss'].isnull()==False, 'before_tss'] = merged.ix[merged['before_tss'].isnull()==False,'coordinate']
    merged['after_tss'] = merged['before_tss']
    merged['before_tss'].fillna(method='ffill', inplace=True)
    merged['after_tss'].fillna(method='bfill',inplace=True)
    merged['dist_to_before_tss'] = np.abs(merged['coordinate']-merged['before_tss'])
    merged['dist_to_after_tss'] = np.abs(merged['coordinate']-merged['after_tss'])
    merged['tss'] = None
    before_ix = (merged['dist_to_before_tss'] < merged['dist_to_after_tss']) | (merged['dist_to_after_tss'].isnull())
    merged.ix[before_ix,'tss'] = merged.ix[before_ix,'before_tss']
    after_ix = (merged['dist_to_before_tss'] >= merged['dist_to_after_tss']) | (merged['dist_to_before_tss'].isnull())
    merged.ix[after_ix,'tss'] = merged.ix[after_ix,'after_tss']
    merged['dist_to_nearest_tss'] = np.abs(merged['coordinate']-merged['tss'])
    merged.drop(['before_tss','after_tss','tss','dist_to_before_tss','dist_to_after_tss'],axis=1,inplace=True)
    merged.dropna(axis=0,inplace=True)
    return merged

def read_WGBS(file):
    bed = pd.read_csv(file,usecols=[0,1,2,5,9,10],header=None,names=['chr','pos1','pos2','strand','total','percent'],sep='\s+')
    bed['coordinate'] = np.where(bed['strand']=='+',bed['pos1'],bed['pos1']-1) ##read 0-based WGBS bed, merge +/- strand
    bed.drop(['pos1','pos2'],axis=1,inplace=True)
    bed['count'] = np.round(bed['total']*bed['percent']/100.0)
    bed.drop(['total','percent'],axis=1,inplace=True)
    bed = bed.groupby(['chr','coordinate']).aggregate({'count':sum}).reset_index()
    
    #    bed_counts = bed.groupby(['chr','coordinate']).aggregate({'count':sum})
    return bed

def hg38tohg19(row):
    global lo
    hg19 = lo.convert_coordinate('chr'+str(row[1]['chr']),row[1]['start'])
    if(len(hg19)>0):
        Chr,start,strand,score = hg19[0]
        try:
            Chr = int(Chr[3:])
        except ValueError:
            if Chr[3:] == 'X':
                Chr = 23
            elif Chr[3:] == 'Y':
                Chr = 24
        start = start+1 ##convert to 1-bbased WGBS coordinate
        end = start
        return [Chr,start,end,row[1]['chr'],row[1]['start']]
    else:
        return [None,None,None,row[1]['chr'],row[1]['start']]

In [3]:
dataset = 'WGBS'
win_path= home+'data/commons/wins.txt'
chrs=np.arange(1,22,dtype='int64')
wins = get_winid.read_wins(win_path,chrs)
all_wgbs_sites_file = home+'data/'+dataset+'/all_wgbs_sites_winid.csv'
hg19_wgbs_file = home+'data/'+dataset+'/hg19_WGBS.csv'

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21']


/home/ec2-user/xiaobo/git/EnsembleCpG/code/features_preprocess/get_winid.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['chr'] = data['chr'].apply(lambda x: int(x[3:]))


In [4]:
###get all WGBS sites only need to run once
data_dir = extra_storage+'WGBS/'
file = data_dir+'ENCFF844EFX.bed'
#wgbs_file = home+'data/'+dataset+'/WGBS.bed'
bed = read_WGBS(file)
bed = get_winid.convert_chr_to_num(bed,chrs).sort_values(['chr','coordinate'])
bed.rename({'coordinate':'start'},axis=1,inplace=True)
bed['end'] = bed['start']+1
bed.drop(['count'],axis=1,inplace=True)
#bed.to_csv(wgbs_file,columns=['chr','start','end'],index=False,sep="\t")

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21']


/home/ec2-user/xiaobo/git/EnsembleCpG/code/features_preprocess/get_winid.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['chr'] = data['chr'].apply(lambda x: int(x[3:]))


In [5]:
##convert to hg19, only need run once
lo = LiftOver('hg38', 'hg19')
coord_hg19 = [hg38tohg19(row)for row in bed.iterrows()]
coord_hg19 = pd.DataFrame(coord_hg19,columns=['chr','coordinate','end','hg38chr','hg38coordinate']).query('chr in @chrs')
coord_hg19.dropna().drop_duplicates(['chr','coordinate']).to_csv(hg19_wgbs_file,index=False)

In [6]:
#using WGBS(hg19) sites only run once
hg19_wgbs = pd.read_csv(hg19_wgbs_file,usecols=[0,1,3,4]).sort_values(['hg38chr','hg38coordinate']).reset_index(drop=True)
#hg19_wgbs = get_winid.convert_chr_to_num(hg19_wgbs,chrs)
all_sites = get_winid.get_winid(wins,hg19_wgbs,True).dropna()
all_sites['winid'] = all_sites['winid'].astype('i8')
all_sites.to_csv(all_wgbs_sites_file,index=False)

In [4]:
all_sites = pd.read_csv(all_wgbs_sites_file)
chrs = all_sites['chr'].unique()
cols=['chr', 'coordinate','strand']
tss =  pd.read_csv(home+'data/commons/tss.txt',sep='\s+',header=None,names=cols,skiprows=1)
tss = get_winid.convert_chr_to_num(tss,chrs)

['1', '16', '15', '2', '19', '12', '5', '11', '21', '18', '4', '17', '14', '7', '3', '9', '20', '8', '10', '6', '13']


/home/ec2-user/xiaobo/git/EnsembleCpG/code/features_preprocess/get_winid.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['chr'] = data['chr'].apply(lambda x: int(x[3:]))


In [5]:
selected_wgbs_tss = wgbs_sites_selection(tss,all_sites)
with pd.HDFStore(home+'data/'+dataset+'/all_selected_wgbs_sites','w') as h5s:
    h5s['all_wgbs'] = selected_wgbs_tss

/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel/__main__.py:15: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel/__main__.py:16: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/si

In [8]:
selected_wgbs_tss

,winid,chr,coordinate,tss_chr,tss_coordinate
0,53.0,1,10469.0,1,11874
1,53.0,1,10471.0,1,11874
2,53.0,1,10484.0,1,11874
3,53.0,1,10489.0,1,11874
4,53.0,1,10493.0,1,11874
5,53.0,1,10497.0,1,11874
6,53.0,1,10525.0,1,11874
7,53.0,1,10542.0,1,11874
8,53.0,1,10563.0,1,11874
9,53.0,1,10571.0,1,11874


In [5]:
start_pos = 0
end_pos = len(selected_wgbs_tss)-1
ranges = np.arange(start_pos,end_pos,2000000)
ranges = np.append(ranges,end_pos)   

In [6]:
start = ranges[0]
end = ranges[1]

In [9]:
selected_wgbs = selected_wgbs_tss[start:end]
sites_file = home+'data/'+dataset+'/all_sites_winid.csv'
additional_feature_file = home+'data/features/'+dataset+'/addtional_features_'+str(start)+'_'+str(end)

In [8]:
from importlib import reload
reload(CADD_Preprocess)

<module 'features_preprocess.CADD_Preprocess' from '/home/ec2-user/xiaobo/git/EnsembleCpG/code/features_preprocess/CADD_Preprocess.py'>

In [ ]:
cadd_preprocess = CADD_Preprocess.CADD_Preprocess(sites_file=sites_file,additional_feature_file=additional_feature_file)
cadd_preprocess.process()
    

/home/ec2-user/xiaobo/git/EnsembleCpG/data/raw/
['1', 88851, 1.781, 1.4426666666666665]
['1', 673949, 3.278, 2.1906666666666665]
['1', 761809, 7.405, 6.674333333333333]
['1', 821249, 3.855, 1.954]
['1', 855741, 5.721, 4.448333333333333]
['1', 875708, 12.17, 11.506666666666666]
['1', 896657, 9.471, 8.707333333333333]
['1', 916990, 2.512, 1.7293333333333336]
['1', 941315, 2.93, 2.305]
['1', 968296, 8.711, 7.131666666666667]
['1', 985916, 24.9, 18.893333333333334]
['1', 1007095, 0.237, 0.156]
['1', 1029544, 3.424, 2.459]
['1', 1061185, 1.6, 1.2653333333333334]
['1', 1085562, 6.323, 5.284]
['1', 1109961, 9.257, 7.8116666666666665]
['1', 1140664, 5.568, 4.521333333333334]
['1', 1162819, 6.275, 5.646666666666666]
['1', 1181935, 15.28, 5.827000000000001]
['1', 1209190, 11.25, 10.618333333333332]
['1', 1229863, 15.63, 13.85]
['1', 1249550, 0.641, 0.5066666666666667]
['1', 1273508, 13.95, 10.115666666666666]
['1', 1291345, 0.163, 0.11733333333333333]
['1', 1323628, 0.849, 0.6863333333333334]
['

In [ ]:
start_pos = 0
end_pos = len(selected_wgbs_tss)-1
ranges = np.arange(start_pos,end_pos,2000000)

tracker = pd.DataFrame({'start':ranges,'1806features':np.zeros_like(ranges),'wgbs':np.zeros_like(ranges),'atac':np.zeros_like(ranges),
                       'rnaseq':np.zeros_like(ranges),'cadd':np.zeros_like(ranges),'dann':np.zeros_like(ranges),
                        'eigen':np.zeros_like(ranges),'genocanyon':np.zeros_like(ranges),'gwava':np.zeros_like(ranges)})
tracker = tracker.set_index('start')
start = ranges[0]
end = ranges[1]
tracker.loc[start,'1806features'] = 1
tracker.loc[start,'wgbs'] = 1
tracker.loc[start,'atac'] = 1
tracker.loc[start,'rnaseq'] = 1
tracker.loc[start,'cadd'] = 1
tracker.to_pickle(home+'data/'+dataset+'tracker.pkl')

In [ ]:
all_wgbs_sites = False
if all_wgbs_sites:
    selected_wgbs_tss = pd.read_csv(all_wgbs_sites_file)
else:
    with pd.HDFStore(home+'data/'+dataset+'/all_selected_wgbs_sites','r') as h5s:
        selected_wgbs_tss = h5s['all_wgbs'] 
start_pos = 0
end_pos = len(selected_wgbs_tss)-1
ranges = np.arange(start_pos,end_pos,2000000)
tracker = pd.DataFrame({'start':ranges,'1806features':np.zeros_like(ranges),'wgbs':np.zeros_like(ranges),'atac':np.zeros_like(ranges),
                       'rnaseq':np.zeros_like(ranges),'cadd':np.zeros_like(ranges),'dann':np.zeros_like(ranges),
                        'eigen':np.zeros_like(ranges),'genocanyon':np.zeros_like(ranges),'gwava':np.zeros_like(ranges)})
tracker = tracker.set_index('start')
tracker.to_pickle(home+'data/'+dataset+'tracker.pkl')
ranges = np.append(ranges,end_pos)    

for i in np.arange(len(ranges)-1):
    start = ranges[i]
    end = ranges[i+1]
    print("start processing range %d to %d"%(start,end))
    selected_wgbs = selected_wgbs_tss[start:end]
    sites_file = home+'data/'+dataset+'/all_sites_winid.csv'
    selected_wgbs.to_csv(sites_file,index=False)
    selected_wgbs.to_csv(home+'data/'+dataset+'/selected_pos_winid.csv',columns=['winid'],index=False,header=None)
    additional_feature_file = home+'data/features/'+dataset+'/addtional_features_'+str(start)+'_'+str(end)
    ###1806 features
    tracker = pd.read_pickle(home+'data/'+dataset+'tracker.pkl')
    if tracker.loc[start,'1806features'] == 1:
        print("1806 features "+str(start)+"_"+str(end)+" already processed")
    else:
        subprocess.call([home+'code/features_preprocess/Feature_export.R',home+'data',dataset,'False'])
        tracker.loc[start,'1806features'] = 1
        tracker.to_pickle(home+'data/'+dataset+'tracker.pkl')
        gc.collect()
        
    
    ###WGBS 
    WGBS_h5s = home+'data/commons/WGBS_single_H5S'
    tracker = pd.read_pickle(home+'data/'+dataset+'tracker.pkl')
    if tracker.loc[start,'wgbs'] == 1:
        print("WGBS features for "+str(start)+"_"+str(end)+" already processed")
    else:
        WGBS_proc = WGBS_preprocess.WGBS_Preprocess(h5s_file=WGBS_h5s,data_dir=extra_storage+'WGBS/',sites_file=sites_file,additional_feature_file=additional_feature_file,hg19_file= home+'data/WGBS/hg19_WGBS.csv')
        if not os.path.exists(WGBS_h5s):
            WGBS_proc.process()
        WGBS_proc.scores()
        tracker.loc[start,'wgbs'] = 1
        tracker.to_pickle(home+'data/'+dataset+'tracker.pkl')
        gc.collect()
    
    
    ###ATAC
    ATAC_h5s = home+'data/commons/ATAC_H5S'
    tracker = pd.read_pickle(home+'data/'+dataset+'tracker.pkl')
    if tracker.loc[start,'atac'] == 1:
        print("ATAC for "+str(start)+"_"+str(end)+" already processed")
    else:
        if os.path.exists(ATAC_h5s):
            atac_process = BED_Preprocess.BED_Preprocessing(h5s_file=ATAC_h5s,sites_file=sites_file,additional_feature_file=additional_feature_file,data_type='ATAC')
            atac_process.process()
        else:
            atac_binning = BED_binning.BED_binning(data_type='ATAC',data_dir=extra_storage+'ATAC/',output=ATAC_h5s,sorted=True)
            atac_binning.binning()
            atac_process = BED_Preprocess.BED_Preprocessing(h5s_file=ATAC_h5s,sites_file=sites_file,additional_feature_file=additional_feature_file,data_type='ATAC')
            atac_process.process()
        tracker.loc[start,'atac'] = 1
        tracker.to_pickle(home+'data/'+dataset+'tracker.pkl')
        gc.collect()
        
    
    ###RNASeq
    RNASeq_h5s = home+'data/RNASeq/'
    tracker = pd.read_pickle(home+'data/'+dataset+'tracker.pkl')
    if tracker.loc[start,'rnaseq'] == 1:
        print("RNASeq for "+str(start)+"_"+str(end)+" already processed")
    else:
        if len(os.listdir(RNASeq_h5s))>0:
            rnaseq_process = BED_Preprocess.BED_Preprocessing(h5s_file=RNASeq_h5s,sites_file=sites_file,additional_feature_file=additional_feature_file, data_type='RNASeq')
            rnaseq_process.process()
        else:
            subprocess.call(['python',home+'code/feature_preprocess/RNASeq_binning.py'])
            rnaseq_process = BED_Preprocess.BED_Preprocessing(h5s_file=RNASeq_h5s,sites_file=sites_file,additional_feature_file=additional_feature_file, data_type='RNASeq')
            rnaseq_process.process()
        tracker.loc[start,'rnaseq'] = 1
        tracker.to_pickle(home+'data/'+dataset+'tracker.pkl')
        gc.collect()
    
    ###CADD
    tracker = pd.read_pickle(home+'data/'+dataset+'tracker.pkl')
    if tracker.loc[start,'cadd'] == 1:
        print("CADD for "+str(start)+"_"+str(end)+" already processed")
    else:    
        cadd_preprocess = CADD_Preprocess.CADD_Preprocess(sites_file=sites_file,additional_feature_file=additional_feature_file)
        cadd_preprocess.process()
        tracker.loc[start,'cadd'] = 1
        tracker.to_pickle(home+'data/'+dataset+'tracker.pkl')
        gc.collect()
    
    ###DANN
    tracker = pd.read_pickle(home+'data/'+dataset+'tracker.pkl')
    if tracker.loc[start,'dann'] == 1:
        print("DANN for "+str(start)+"_"+str(end)+" already processed")
    else: 
        dann_preprocess = DANN_Preprocess.DANN_Preprocess(sites_file=sites_file,additional_feature_file=additional_feature_file)
        dann_preprocess.process()
        tracker.loc[start,'dann'] = 1
        tracker.to_pickle(home+'data/'+dataset+'tracker.pkl')
        gc.collect()
    
    
    ###eigen
    tracker = pd.read_pickle(home+'data/'+dataset+'tracker.pkl')
    if tracker.loc[start,'eigen'] == 1:
        print("Eigen for "+str(start)+"_"+str(end)+" already processed")
    else: 
        eigen_preprocess = Eigen_Preprocess.Eigen_Preprocess(sites_file=sites_file,additional_feature_file=additional_feature_file)
        eigen_preprocess.process()        
        tracker.loc[start,'eigen'] = 1
        tracker.to_pickle(home+'data/'+dataset+'tracker.pkl')
        gc.collect()
    
    ###genocanyon
    tracker = pd.read_pickle(home+'data/'+dataset+'tracker.pkl')
    if tracker.loc[start,'genocanyon'] == 1:
        print("genocanyon for "+str(start)+"_"+str(end)+" already processed")
    else:      
        genocanyon_scores = extra_storage+'GenoCanyon/Results/'+dataset+'/selected_site_scores.txt'
        data_dir=extra_storage+'GenoCanyon/Results/'+dataset+'/'
        if os.path.exists(genocanyon_scores):
            genocanyon_preprocess = GenoCanyon_Preprocess.GenoCanyon_Preprocess(data_dir=data_dir,sites_file=sites_file,additional_feature_file=additional_feature_file)
            genocanyon_preprocess.process('selected_site_scores.txt')
        else:
            print('Running GenoCanyon R script...')
            subprocess.call([home+'code/features_preprocess/GenoCanyon_Preprocess.R',"FALSE",home,extra_storage,dataset])
        tracker.loc[start,'genocanyon'] = 1
        tracker.to_pickle(home+'data/'+dataset+'tracker.pkl')
        gc.collect()
    
    ###gwava
    tracker = pd.read_pickle(home+'data/'+dataset+'tracker.pkl')
    if tracker.loc[start,'gwava'] == 1:
        print("GWAVA for "+str(start)+"_"+str(end)+" already processed")
    else:
        gwava_preprocess = GWAVA_Preprocess.GWAVA_Preprocess(sites_file=sites_file,additional_feature_file=additional_feature_file)
        gwava_preprocess.process()
        tracker.loc[start,'gwava'] = 1
        tracker.to_pickle(home+'data/'+dataset+'tracker.pkl')
        gc.collect()        
    
    selected_wgbs = pd.read_csv(home+'data/'+dataset+'/all_sites_winid.csv')
    feature_dir = home+'data/features/'+dataset+'/'
    files = os.listdir(feature_dir)
    pattern = '.*all.csv$'
    reg = re.compile(pattern)
    files = [name for name in files if len(reg.findall(name))>0]
    
    for file in files:    
        feature = pd.read_csv(feature_dir+file)
        print(len(feature.columns))
        selected_wgbs = pd.concat([selected_wgbs,feature],axis=1)
    
    rename_features(selected_wgbs)
    additional_features = ['ATAC','CADD','DANN','Eigen','GenoCanyon','RNASeq','WGBS','GWAVA']
    
    #merge with additional features
    with pd.HDFStore(additional_feature_file,'r') as h5s:
        for feature in additional_features:
            feature_frame = h5s[feature]
            selected_wgbs = pd.concat([selected_wgbs,feature_frame],axis=1)
    selected_wgbs = selected_wgbs.loc[:,~selected_wgbs.columns.duplicated()]
    selected_wgbs['chr'] = selected_wgbs['chr'].astype('i8')
    
    #nearest tss distance    
    chrs = selected_wgbs['chr'].unique()
    cols=['chr', 'coordinate','strand']
    tss =  pd.read_csv(home+'data/commons/tss.txt',sep='\s+',header=None,names=cols,skiprows=1)
    tss = get_winid.convert_chr_to_num(tss,chrs)
    tss.sort_values(['chr','coordinate'],inplace=True)
    selected_wgbs = nearest_tss(tss,selected_wgbs)
    with pd.HDFStore(home+'data/'+dataset+'/all_features_'+str(start)+'_'+str(end),'w') as h5s:
        h5s['all_features'] = selected_wgbs
    
    gc.collect()

In [15]:
selected_wgbs


,winid,chr,coordinate,tss_chr,tss_coordinate,A549,Astrocy,Colon_OC,Endometrium_OC,Frontal_cortex_OC,...,NCFF795DNO_WGBS_counts,NCFF801OHX_WGBS_counts,NCFF811QOG_WGBS_counts,NCFF831OYO_WGBS_counts,NCFF843SYR_WGBS_counts,NCFF847OWL_WGBS_counts,NCFF874GGB_WGBS_counts,NCFF913ZNZ_WGBS_counts,NCFF923CZC_WGBS_counts,dist_to_nearest_tss
0,53.0,1,10469.0,1.0,11874.0,5.0,1.0,43.0,43.0,16.0,...,0.0,70.0,0.0,4.0,12.0,13.0,22.0,5.0,2.0,1405.0
1,53.0,1,10471.0,1.0,11874.0,5.0,1.0,43.0,43.0,16.0,...,0.0,73.0,2.0,8.0,15.0,13.0,21.0,6.0,3.0,1403.0
2,53.0,1,10484.0,1.0,11874.0,5.0,1.0,43.0,43.0,16.0,...,0.0,70.0,6.0,7.0,18.0,14.0,20.0,10.0,2.0,1390.0
3,53.0,1,10489.0,1.0,11874.0,5.0,1.0,43.0,43.0,16.0,...,0.0,70.0,3.0,7.0,25.0,15.0,19.0,11.0,4.0,1385.0
4,53.0,1,10493.0,1.0,11874.0,5.0,1.0,43.0,43.0,16.0,...,0.0,75.0,7.0,7.0,17.0,17.0,27.0,12.0,3.0,1381.0
5,53.0,1,10497.0,1.0,11874.0,5.0,1.0,43.0,43.0,16.0,...,0.0,88.0,6.0,8.0,27.0,18.0,26.0,14.0,4.0,1377.0
6,53.0,1,10525.0,1.0,11874.0,5.0,1.0,43.0,43.0,16.0,...,0.0,118.0,12.0,11.0,32.0,23.0,32.0,17.0,4.0,1349.0
7,53.0,1,10542.0,1.0,11874.0,5.0,1.0,43.0,43.0,16.0,...,0.0,101.0,11.0,11.0,35.0,23.0,25.0,22.0,9.0,1332.0
8,53.0,1,10563.0,1.0,11874.0,5.0,1.0,43.0,43.0,16.0,...,0.0,113.0,16.0,11.0,37.0,22.0,21.0,26.0,9.0,1311.0
9,53.0,1,10571.0,1.0,11874.0,5.0,1.0,43.0,43.0,16.0,...,0.0,132.0,20.0,11.0,39.0,17.0,30.0,30.0,11.0,1303.0
